In [1]:
import pandas as pandas
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import time
import os
import random
sns.set()

## Generación de prueba

In [ ]:
image_path = "../datos_entrada/ej1/"
fileName="tests.out"
testCases="testCases.in"
test_times = 10
images = ["test1.jpg"]
kRange = range(500,6001,500)

for k in kRange:
    for image in images:
        tempFile = ("{0}_{1}").format(image,k)
        with open(tempFile, 'w') as f:
            f.write(str(n) + " " + str(W) + "\n")
            for i in range(1,n+1):
                f.write(str(random.randint(1, W)) + " " + str(random.randint(1, 300)) + "\n")
            f.write("1 0 0 0")
            f.flush()
